In [1]:
# Carregar llibreries

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import numpy as np
import random
import time
from google.colab import files

In [2]:
import ipywidgets as widgets
from IPython.display import display

# Crear commutador de botons
commutador = widgets.ToggleButtons(
    options=['Alta i caducats', 'Tots'],
    description='Selecciona:',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Selecciona Alta i caducats', 'Selecciona Tots'],
)

tria = None

# Funció per gestionar la tria de l'usuari
def seleccionar_opcio(change):
    global tria
    tria = change['new']

# Assignar funció al commutador
commutador.observe(seleccionar_opcio, names='value')

# Mostrar el commutador
print("Tria una d'aquestes dues opcions:")
display(commutador)


# Carrega el fitxer Excel

# INFORME GPG: Inventari de drets immaterials - Dates de vigència

# nom_fitxer = '/content/Inventaridedretsimmaterials-Datesdevigència (2).xlsx'  # Canvia a la ruta correcta del teu fitxer
# df = pd.read_excel(nom_fitxer, header=13)  # Ajusta el valor de 'header' segons la posició de les capçaleres

print()
print("SELECCIONA INFORME DATES DE VIGÈNCIA GPG")
print()

# Carrega l'arxiu XLSX
uploaded = files.upload()

# Obtenir el nom de l'arxiu carregat
nom_arxiu = list(uploaded.keys())[0]

# Llegir l'arxiu XLSX amb pandas
df = pd.read_excel(nom_arxiu, header=13)  # Ajusta el valor de 'header' segons la posició de les capçaleres

# Selecciona les columnes que necessites
columnes_seleccionades = [
    "Codi registre",
    "Registre d'inscripció",
    "Data fi vigència",
    "Data inici vigència",
    "Data renovació",
    "Catàleg",
    "Estat",
    "Titular",
    "Número d'inventari"
]

df = df[columnes_seleccionades]

Tria una d'aquestes dues opcions:


ToggleButtons(description='Selecciona:', options=('Alta i caducats', 'Tots'), tooltips=('Selecciona Alta i cad…


SELECCIONA INFORME DATES DE VIGÈNCIA GPG



Saving Dates vigència marques PANGEA.xlsx to Dates vigència marques PANGEA (22).xlsx


In [3]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
46 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
w

In [4]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
import chromedriver_autoinstaller

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # this is must
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)

In [5]:
tria

In [6]:
# Filtrar XLSX

# Obtenir la data d'avui
avui = datetime.now()

if tria == "Alta i caducats":
  df2 = df[
      (df['Estat'] == 'Alta') &
      (df["Catàleg"] == 'Marca') &
      (df["Registre d'inscripció"] == "Oficina Espanyola de Patents i Marques (OEPM)") &
      (df["Data fi vigència"] < avui)
        ]
  # print(df2["Codi registre"])

else:
  df2 = df[
    (df["Catàleg"] == 'Marca') &
    (df["Registre d'inscripció"] == "Oficina Espanyola de Patents i Marques (OEPM)")
      ]

In [7]:
def separar(cadena):
    # Patró de cerca per la data (dd/mm/yyyy)
    patro_data = re.compile(r'\b(\d{1,2}/\d{1,2}/\d{4})\b')

    # Cerca la data a la cadena
    coincidencia_data = patro_data.search(cadena)

    if coincidencia_data:
        # Si es troba la data, separa el text i la data
        text = cadena[:coincidencia_data.start()].strip()
        data = coincidencia_data.group()
    else:
        # Si no es troba la data, retorna la cadena completa com a text i deixa la data buida
        text = cadena
        data = ""

    return text, data

# Exemple d'ús
# cadena = "PUBL.CONCES.RENOV. DE F.RESOL. 06/11/2023"
# text, data = separar(cadena)


In [8]:
def dades(codi):

    url = "http://consultas2.oepm.es/ceo/jsp/busqueda/consultaExterna.xhtml?numExp=M" + codi

    driver.get(url)

    dades = {}  # Crear un diccionari per emmagatzemar les dades

    dades['Codi'] = codi

    try:
        response = requests.get(url)

        if response.status_code == 200:

          try:

            # Parseja el contingut HTML amb BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')

            # Troba l'element <h4> amb el text "Actos de tramitación:"
            h4_actos = soup.find('h4', string=lambda text: "Actos de tramitación:" in text if text else False)
            h4_actos = h4_actos.find_next('div', class_='datos')

            # Troba tots els elements <td> amb l'atribut role="gridcell"
            td_elements = h4_actos.find_all('td', {'role': 'gridcell'})

            # Verifica si hi ha algun element
            if td_elements:
                # Obtenir l'últim element i extreure'n el text
                dades['last_td'] = td_elements[-1].get_text(strip=True)
                dades['Data publicació'] = td_elements[-2].get_text(strip=True)

                dades['Anotació'], dades['Data anotació'] = separar(dades['last_td'])

            # Busca l'element <h4> que contingui el text "Fecha presentación solicitud otorgada:"
            presentacio_h4 = soup.find('h4', string=lambda text: "Fecha presentación solicitud otorgada:" in text if text else False)

            # Obte el text de l'element següent (en aquest cas, l'element <p>)
            if presentacio_h4:
                dades['Data de presentació'] = presentacio_h4.find_next('p').get_text(strip=True)

            # Busca l'element <h4> que contingui el text "Estado:"
            estado_h4 = soup.find('h4', string=lambda text: "Estado:" in text if text else False)

            # Obte el text de l'element següent (en aquest cas, l'element <p>)
            if estado_h4:
                dades['Estat'] = estado_h4.find_next('p').get_text(strip=True)

            # Troba l'element <div> amb l'id "marco-imagen-sitamar"
            div_marco_imagen = soup.find('div', {'id': 'marco-imagen-sitamar'})

            if div_marco_imagen:
                # Troba l'element <img> dins del div
                img_element = div_marco_imagen.find('img')

                if img_element:
                    # Obté el valor de l'atribut src
                    src_valor = img_element.get('src', '')

                    # Imprimeix el resultat
                    dades['Imatge'] = "http://consultas2.oepm.es"+src_valor
                else:
                    dades['Imatge'] = ""

            # Troba l'element <h3>
            h3_element = soup.find('h3')

            if h3_element:
                # Obté el text de l'element <h3>
                text_h3 = h3_element.get_text(strip=True)

                # Utilitza una expressió regular per seleccionar el segon grup
                match = re.match(r'(.*)\s-\s(.*)', text_h3)

                if match:
                    segon_grup = match.group(2)
                    dades['Denominació'] = segon_grup

            # Troba l'element <h4>
            h4_element = soup.find('h4', string=lambda text: "Tipo:" in text if text else False)

            if h4_element:
                # Obtenir el text de l'element <h4>
                text_h4 = h4_element.find_next('p').get_text(strip=True)

                dades['Tipus'] = text_h4

            # Extreu el text que segueix a <span class="campo">Nombre:</span>
            nom_span = soup.find('span', class_='campo', string="Nombre:")
            if nom_span:
                text_nom = nom_span.find_next('div', class_='datos-sitamar-divSubRight').text.strip()
                # Comprova si hi ha l'etiqueta <br/>
                if "\n" in text_nom:
                    # Divideix el text en dos blocs amb l'etiqueta <br />
                    motiu, titular = map(str.strip, text_nom.split('\n', 1))
                    dades['Titular OEPM'] = titular
                    dades['Motiu'] = motiu
                else:
                    # Si no hi ha <br />, posa tot el text a 'titular'
                    dades['Titular OEPM'] = text_nom
                    dades['Motiu'] = ""

            dades['Url'] = url
            dades['Titular GPG'] = df2['Titular'][df2['Codi registre'] == float(codi)].values[0]
            dades['Inici vigència GPG'] = pd.to_datetime(df2["Data inici vigència"][df2['Codi registre'] == float(codi)]).dt.date.values[0]
            dades['Fi vigència GPG'] = pd.to_datetime(df2["Data fi vigència"][df2['Codi registre'] == float(codi)]).dt.date.values[0]
            dades['Núm. secció'] = df2["Número d'inventari"][df2['Codi registre'] == float(codi)].values[0]

          except:
              dades['Estat'] = "Núm. registre incorrecte"


          try:
              # Executa JavaScript per fer clic al botó
              driver.execute_script("document.querySelector('[id=\"pestannas\\:tabPagos\"]').click();")
              # Espera fins que la pàgina es carregui (ajusta a les teves necessitats)
              time.sleep(2)

              html_pagos = driver.page_source

              soup_pagos = BeautifulSoup(html_pagos, 'html.parser')

              # Troba totes les files de la taula
              rows = soup_pagos.find_all('tr')

              # Llista per emmagatzemar les dades
              data = []

              # Itera sobre les files de la taula, extreu les dades i les afegeix a la llista
              for row in rows:
                  cells = row.find_all('td')
                  if len(cells) == 4:  # Assegura't que hi ha quatre cel·les
                      fecha = cells[0].text.strip()
                      descripcion = cells[1].text.strip()
                      importe_pagado = cells[2].text.strip()
                      importe_devuelto = cells[3].text.strip()
                      data.append({
                          'Fecha': fecha,
                          'Descripción': descripcion,
                          'Importe pagado': importe_pagado,
                          'Importe devuelto': importe_devuelto
                      })

              dades['Pagament data'] = data[-1]["Fecha"]
              dades["Pagament descripció"] = data[-1]["Descripción"]
              dades['Pagament import'] = data[-1]["Importe pagado"]
              dades['Pagament import retornat'] = data[-1]["Importe devuelto"]

          except:
              print("Error en intentar obrir finestra de pagaments")
              dades['Pagament data'] = "ERROR"
              dades["Pagament descripció"] = "ERROR"
              dades['Pagament import'] = "ERROR"
              dades['Pagament import retornat'] = "ERROR"

        else:
            print(f"S'ha produït un error en la sol·licitud. Codi d'estat: {response.status_code}")

    except requests.RequestException as e:
        print(f"Error de connexió: {e}")

    return dades  # Retorna el diccionari amb les dades

In [9]:
'''
# Exemple d'ús

resultats = dades("2551345")

# Ara pots fer el que vulguis amb les dades obtingudes
if resultats:
    # Fer quelcom amb les dades, com ara imprimir-les
    print("Dades obtingudes:")
    print(f"codi: {resultats.get('Codi')}")
    print(f"denominació: {resultats.get('Denominació')}")
    print(f"estado_text: {resultats.get('Estat')}")
    print(f"data_publicacio: {resultats.get('Data publicació')}")
    print(f"anotacio: {resultats.get('Anotació')}")
    print(f"anotacio_data: {resultats.get('Data anotació')}")
    print(f"imatge: {resultats.get('Imatge')}")
    print(f"titular: {resultats.get('Titular OEPM')}")
    print(f"motiu: {resultats.get('Motiu')}")
    print(f"tipus: {resultats.get('Tipus')}")
    print(f"Url: {resultats.get('Url')}")
'''

'\n# Exemple d\'ús\n\nresultats = dades("2551345")\n\n# Ara pots fer el que vulguis amb les dades obtingudes\nif resultats:\n    # Fer quelcom amb les dades, com ara imprimir-les\n    print("Dades obtingudes:")\n    print(f"codi: {resultats.get(\'Codi\')}")\n    print(f"denominació: {resultats.get(\'Denominació\')}")\n    print(f"estado_text: {resultats.get(\'Estat\')}")\n    print(f"data_publicacio: {resultats.get(\'Data publicació\')}")\n    print(f"anotacio: {resultats.get(\'Anotació\')}")\n    print(f"anotacio_data: {resultats.get(\'Data anotació\')}")\n    print(f"imatge: {resultats.get(\'Imatge\')}")\n    print(f"titular: {resultats.get(\'Titular OEPM\')}")\n    print(f"motiu: {resultats.get(\'Motiu\')}")\n    print(f"tipus: {resultats.get(\'Tipus\')}")\n    print(f"Url: {resultats.get(\'Url\')}")\n'

In [10]:
# Baixar la informació

marques_revisades = pd.DataFrame()

id = df2["Codi registre"]

print("Marques a processar: ", len(id), "\n")

for index, x in enumerate(id):
    print(str(round(index/len(id)*100, 2))+"%")
    resultats = dades(str(x)[:-2])
    marques_revisades = pd.concat([marques_revisades, pd.DataFrame([resultats])], ignore_index=True)
    time.sleep(random.uniform(10, 15))
print("100.00 %")

marques_revisades.drop('last_td', axis=1, inplace=True)
marques_revisades

Marques a processar:  6 

0.0%
16.67%
33.33%
50.0%
66.67%
83.33%
100.00 %


,Codi,Data publicació,Anotació,Data anotació,Data de presentació,Estat,Imatge,Denominació,Tipus,Titular OEPM,Motiu,Url,Titular GPG,Inici vigència GPG,Fi vigència GPG,Núm. secció,Pagament data,Pagament descripció,Pagament import,Pagament import retornat
0,2905002,03/10/2019,PUBL.CONCES.RENOV. DE F.RESOL.,27/09/2019,14/12/2009,En vigor,,AQU CATALUNYA,Denominativa estándar,AGENCIA PER A LA QUALITAT DEL SISTEMA UNIVERSI...,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Agència per a la Qualitat del Sistema Universi...,2009-12-14,2029-12-14,5737,28/08/2019,RENOVACION MARCA O NOMBRE COMERCIAL INTERNET,437.46,0.0
1,2905006,11/12/2020,PUBLIC. CADUCIDAD FECH. RESOL:,02/12/2020,14/12/2009,Caducado,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,A QU AGENCIA PER A LA QUALITAT DEL SISTEMA UNI...,Mixta,AGENCIA PER A LA QUALITAT DEL SISTEMA UNIVERSI...,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Agència per a la Qualitat del Sistema Universi...,2009-12-14,2029-12-14,5739,14/12/2009,SOLIC.INTERNET MARCA O NOMBRE POR CLASE,524.88,0.0
2,2892939,17/06/2019,PUBL.CONCES.RENOV. DE F.RESOL.,11/06/2019,23/09/2009,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,ATM AREA DE BARCELONA AUTORITAT DEL TRANSPORT ...,Mixta,"AUTORITAT DEL TRANSPORT METROPOLITA, CONSORCI ...",,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,"Autoritat del Transport Metropolità, Consorci ...",2009-09-23,2029-09-23,4794,30/05/2019,RENOVACION MARCA O NOMBRE COMERCIAL INTERNET,1022.1,0.0
3,2105927,22/09/2017,PUBL.CONCES.RENOV. DE F.RESOL.,18/09/2017,23/07/1997,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,TRANSMET TRANSPORT METROPOLITA,Mixta,L'AREA DE BARNA AUTORITAT DEL TRANS. METROPOLI...,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,"Autoritat del Transport Metropolità, Consorci ...",1997-07-23,2027-07-23,4795,30/06/2017,RENOVACION MARCA O NOMBRE COMERCIAL INTERNET,143.7,0.0
4,2904984,29/11/2019,PUBL.CONCES.RENOV. DE F.RESOL.,25/11/2019,14/12/2009,En vigor,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,CONCA CONSELL NACIONAL DE LA CULTURA I DE LES ...,Mixta,CONSELL NACIONAL DE LA CULTURA I DE LES ARTS,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Consell Nacional de la Cultura i de les Arts,2009-12-14,2029-12-14,4449,13/11/2019,RENOVACION MARCA O NOMBRE COMERCIAL INTERNET,534.9,0.0
5,2994768,27/10/2023,PUBLIC. CADUCIDAD FECH. RESOL:,20/10/2023,08/08/2011,Caducado,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,DIPLOCAT. CONSELL DE DIPLOMACIA PUBLICA DE CAT...,Mixta,PATRONAT CATALUNYA MON,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Patronat Catalunya Món - Consell de Diplomàcia...,2011-08-08,2021-08-08,4231,24/08/2011,SOLIC.REGISTRO INTERNACIONAL,41.43,0.0


In [11]:
import pandas as pd

# Defineix l'ordre desitjat de les columnes
ordre_columnes = ['Codi', 'Núm. secció', 'Denominació', 'Tipus', 'Estat', 'Data de presentació', 'Data publicació', 'Anotació', 'Data anotació', 'Titular OEPM', 'Motiu', 'Imatge', 'Url', 'Titular GPG', 'Inici vigència GPG', 'Fi vigència GPG', 'Pagament data', 'Pagament descripció', 'Pagament import', 'Pagament import retornat']

# Suposant que marques_revisades ja està definit i té les mateixes columnes que ordre_columnes
# Reordena les columnes del DataFrame original
marques_revisades = marques_revisades[ordre_columnes]
marques_revisades = marques_revisades.fillna("")

marques_revisades

,Codi,Núm. secció,Denominació,Tipus,Estat,Data de presentació,Data publicació,Anotació,Data anotació,Titular OEPM,Motiu,Imatge,Url,Titular GPG,Inici vigència GPG,Fi vigència GPG,Pagament data,Pagament descripció,Pagament import,Pagament import retornat
0,2905002,5737,AQU CATALUNYA,Denominativa estándar,En vigor,14/12/2009,03/10/2019,PUBL.CONCES.RENOV. DE F.RESOL.,27/09/2019,AGENCIA PER A LA QUALITAT DEL SISTEMA UNIVERSI...,,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Agència per a la Qualitat del Sistema Universi...,2009-12-14,2029-12-14,28/08/2019,RENOVACION MARCA O NOMBRE COMERCIAL INTERNET,437.46,0.0
1,2905006,5739,A QU AGENCIA PER A LA QUALITAT DEL SISTEMA UNI...,Mixta,Caducado,14/12/2009,11/12/2020,PUBLIC. CADUCIDAD FECH. RESOL:,02/12/2020,AGENCIA PER A LA QUALITAT DEL SISTEMA UNIVERSI...,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Agència per a la Qualitat del Sistema Universi...,2009-12-14,2029-12-14,14/12/2009,SOLIC.INTERNET MARCA O NOMBRE POR CLASE,524.88,0.0
2,2892939,4794,ATM AREA DE BARCELONA AUTORITAT DEL TRANSPORT ...,Mixta,En vigor,23/09/2009,17/06/2019,PUBL.CONCES.RENOV. DE F.RESOL.,11/06/2019,"AUTORITAT DEL TRANSPORT METROPOLITA, CONSORCI ...",,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,"Autoritat del Transport Metropolità, Consorci ...",2009-09-23,2029-09-23,30/05/2019,RENOVACION MARCA O NOMBRE COMERCIAL INTERNET,1022.1,0.0
3,2105927,4795,TRANSMET TRANSPORT METROPOLITA,Mixta,En vigor,23/07/1997,22/09/2017,PUBL.CONCES.RENOV. DE F.RESOL.,18/09/2017,L'AREA DE BARNA AUTORITAT DEL TRANS. METROPOLI...,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,"Autoritat del Transport Metropolità, Consorci ...",1997-07-23,2027-07-23,30/06/2017,RENOVACION MARCA O NOMBRE COMERCIAL INTERNET,143.7,0.0
4,2904984,4449,CONCA CONSELL NACIONAL DE LA CULTURA I DE LES ...,Mixta,En vigor,14/12/2009,29/11/2019,PUBL.CONCES.RENOV. DE F.RESOL.,25/11/2019,CONSELL NACIONAL DE LA CULTURA I DE LES ARTS,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Consell Nacional de la Cultura i de les Arts,2009-12-14,2029-12-14,13/11/2019,RENOVACION MARCA O NOMBRE COMERCIAL INTERNET,534.9,0.0
5,2994768,4231,DIPLOCAT. CONSELL DE DIPLOMACIA PUBLICA DE CAT...,Mixta,Caducado,08/08/2011,27/10/2023,PUBLIC. CADUCIDAD FECH. RESOL:,20/10/2023,PATRONAT CATALUNYA MON,,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Patronat Catalunya Món - Consell de Diplomàcia...,2011-08-08,2021-08-08,24/08/2011,SOLIC.REGISTRO INTERNACIONAL,41.43,0.0


In [12]:
# Crea columna ACCIÓ aplicant condicions

from datetime import timedelta

def process_row(row):
    date_format = "%d/%m/%Y"
    if row['Inici vigència GPG']:
      inici_vigencia = row['Inici vigència GPG'].strftime(date_format)

      if row['Estat'] == "Núm. registre incorrecte":
        return "Núm. registre incorrecte"
      elif row['Data de presentació'] != inici_vigencia:
          return "Dates no coincidents"
      elif row['Estat'] == "Caducado":
          return "Baixa"
      elif row['Data anotació'] == "":
          fi_vigencia = row['Fi vigència GPG']
          if fi_vigencia < (datetime.now().date() - timedelta(days=365)):
              return "Més d'un any caducada"
          else:
              return "Deixar pendent"
      elif datetime.strptime(row['Data anotació'], date_format).date() > row['Fi vigència GPG']:
          return "Renovat"
      elif row['Fi vigència GPG'] < (datetime.now().date() - timedelta(days=365)):
          return "Més d'un any caducada"
      else:
          return "Deixar pendent"

# Aplica la funció a cada fila del DataFrame
marques_revisades['Acció'] = marques_revisades.apply(process_row, axis=1)

# Mostra el DataFrame amb la nova columna
marques_revisades


,Codi,Núm. secció,Denominació,Tipus,Estat,Data de presentació,Data publicació,Anotació,Data anotació,Titular OEPM,...,Imatge,Url,Titular GPG,Inici vigència GPG,Fi vigència GPG,Pagament data,Pagament descripció,Pagament import,Pagament import retornat,Acció
0,2905002,5737,AQU CATALUNYA,Denominativa estándar,En vigor,14/12/2009,03/10/2019,PUBL.CONCES.RENOV. DE F.RESOL.,27/09/2019,AGENCIA PER A LA QUALITAT DEL SISTEMA UNIVERSI...,...,,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Agència per a la Qualitat del Sistema Universi...,2009-12-14,2029-12-14,28/08/2019,RENOVACION MARCA O NOMBRE COMERCIAL INTERNET,437.46,0.0,Deixar pendent
1,2905006,5739,A QU AGENCIA PER A LA QUALITAT DEL SISTEMA UNI...,Mixta,Caducado,14/12/2009,11/12/2020,PUBLIC. CADUCIDAD FECH. RESOL:,02/12/2020,AGENCIA PER A LA QUALITAT DEL SISTEMA UNIVERSI...,...,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Agència per a la Qualitat del Sistema Universi...,2009-12-14,2029-12-14,14/12/2009,SOLIC.INTERNET MARCA O NOMBRE POR CLASE,524.88,0.0,Baixa
2,2892939,4794,ATM AREA DE BARCELONA AUTORITAT DEL TRANSPORT ...,Mixta,En vigor,23/09/2009,17/06/2019,PUBL.CONCES.RENOV. DE F.RESOL.,11/06/2019,"AUTORITAT DEL TRANSPORT METROPOLITA, CONSORCI ...",...,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,"Autoritat del Transport Metropolità, Consorci ...",2009-09-23,2029-09-23,30/05/2019,RENOVACION MARCA O NOMBRE COMERCIAL INTERNET,1022.1,0.0,Deixar pendent
3,2105927,4795,TRANSMET TRANSPORT METROPOLITA,Mixta,En vigor,23/07/1997,22/09/2017,PUBL.CONCES.RENOV. DE F.RESOL.,18/09/2017,L'AREA DE BARNA AUTORITAT DEL TRANS. METROPOLI...,...,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,"Autoritat del Transport Metropolità, Consorci ...",1997-07-23,2027-07-23,30/06/2017,RENOVACION MARCA O NOMBRE COMERCIAL INTERNET,143.7,0.0,Deixar pendent
4,2904984,4449,CONCA CONSELL NACIONAL DE LA CULTURA I DE LES ...,Mixta,En vigor,14/12/2009,29/11/2019,PUBL.CONCES.RENOV. DE F.RESOL.,25/11/2019,CONSELL NACIONAL DE LA CULTURA I DE LES ARTS,...,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Consell Nacional de la Cultura i de les Arts,2009-12-14,2029-12-14,13/11/2019,RENOVACION MARCA O NOMBRE COMERCIAL INTERNET,534.9,0.0,Deixar pendent
5,2994768,4231,DIPLOCAT. CONSELL DE DIPLOMACIA PUBLICA DE CAT...,Mixta,Caducado,08/08/2011,27/10/2023,PUBLIC. CADUCIDAD FECH. RESOL:,20/10/2023,PATRONAT CATALUNYA MON,...,http://consultas2.oepm.es/ceo/ImagenMarcaServl...,http://consultas2.oepm.es/ceo/jsp/busqueda/con...,Patronat Catalunya Món - Consell de Diplomàcia...,2011-08-08,2021-08-08,24/08/2011,SOLIC.REGISTRO INTERNACIONAL,41.43,0.0,Baixa


In [13]:
marques_revisades.to_excel(f'marques_oepm_revisades_{avui.year}{avui.month:02d}{avui.day:02d}.xlsx', index=False)

In [14]:
files.download(f'marques_oepm_revisades_{avui.year}{avui.month:02d}{avui.day:02d}.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>